In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from skimage.transform import rescale
from waveletUtil import *
from ipywidgets import VBox, IntSlider
from skimage.util import view_as_blocks
from skimage.util import montage 


In [2]:
plt.figure()
I = plt.imread('original.jpg')
I = I/I.max()
I = I[...,:3].copy()
I =I[:1152, :1152, :3] #crop
plt.imshow(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
Ib = view_as_blocks(I, block_shape = (8,8,3))

In [4]:
Ib = np.squeeze(Ib)

blockView = Ib.reshape([Ib.shape[0]*Ib.shape[1]] + list(Ib.shape[2:]))

In [5]:
newImageBlocks = np.zeros(blockView.shape)

for i, block in enumerate(blockView):
    bT = np.mean(block, axis=(0,1)) # Some transform of the block
    # bT is a (3,) array of the average color of the block
    # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
    newImageBlocks[i][:] = np.reshape(bT, (1,1,3)) 

In [6]:
I_mean = montage(newImageBlocks, grid_shape=[Ib.shape[0], Ib.shape[1]], multichannel=True) 

In [7]:
f, ax = plt.subplots(1,2, figsize=(10,10), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('Original Image')
ax[1].imshow(I_mean)
ax[1].set_title('Mean Reconstruction')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
sI = rescale(I, 1/3, order = 1, anti_aliasing=False, multichannel=True)
plt.figure()
plt.imshow(sI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
Ib = view_as_blocks(sI, block_shape = (8,8,3))

In [10]:
Ib.shape

(48, 48, 1, 8, 8, 3)

In [11]:
H = makeHaarMatrix(8)

In [12]:
#The transform image, an image of coefficients wrt the Haar basis.
def generateCoef(sI, H):
    CT = np.zeros(sI.shape)
    for chan in range(3):
        CT[...,chan] = np.matmul(H, np.matmul(sI[...,chan], H.transpose()))
    return CT

# #RI will represent the reconstructed image as we add back more
# #Haar patterns
# RI = np.zeros(sI.shape) #Still should be 8x8

In [13]:
# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(8), np.arange(8), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)

In [14]:
xs1 = np.meshgrid(np.arange(48), np.arange(48), indexing='ij')
coords1 = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs1]], axis=1)

In [15]:
def make_image(change, H, CT, sI):
    global coords
    ReI = np.zeros(sI.shape)
    Bij = np.outer(H[0, :], H[0, :])
    for ind in range(change):
        i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
        Bij = np.outer(H[i, :], H[j, :])
    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
        for chan in range(3):
            ReI[...,chan] += CT[i,j,chan]*Bij
    return Bij, ReI

In [16]:
def mergeImg(change, Ib, H):
    global coords1
    Ic = np.squeeze(Ib)

    blockView = Ic.reshape([Ic.shape[0]*Ic.shape[1]] + list(Ic.shape[2:]))
    newImageBlocks = np.zeros(blockView.shape)
    
    for ind, block in enumerate(blockView):
        i, j = coords1[ind]
        base, bT = make_image(change, H, generateCoef(Ib[i,j,0], H), Ib[i,j,0]) # Some transform of the block
    # bT is a (3,) array of the average color of the block
    # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
        newImageBlocks[ind][:] = bT
    I_new = montage(newImageBlocks[:,:], grid_shape=[Ib.shape[0], Ib.shape[1]], multichannel=True)
    return I_new

In [17]:
basis = np.outer(H[0, :], H[0, :])
init = mergeImg(1, Ib, H)
plt.ioff()
plt.clf()

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#FloatLogSlider
slider = IntSlider(
    orientation='horizontal',
    value=1,
    min=1,
    max=64,
    step=1,
    description='Base element'
)

fig_args = {'num':' ', 'frameon':True}
fig, ax = plt.subplots(1,3, figsize=(10,3), **fig_args) 

# display artists I'll update
adisp = ax[0].imshow(basis, cmap='gray', vmin=-0.5, vmax=0.5)
hdisp = ax[1].imshow(init, cmap='magma', vmin=0, vmax=1)
ax[2].imshow(sI)
ax[2].set_title("OG")

def update_image(change):
    global H, coords
    i, j = coords[change.new-1]
    new_base = np.outer(H[i, :], H[j, :])
    new_img = mergeImg(change.new, Ib, H)
    hdisp.set_array(new_img)
    # need to reset the color limits each time since gaus_h is changing a lot. 
    hdisp.set_clim(0,1) 
    
    adisp.set_array(new_base)
    
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])